# Introduction to the xgbsurv package - Accelerated Hazards

This notebook introduces `xgbsurv` using a specific dataset. It structured by the following steps:

- Load data
- Load model
- Fit model
- Predict and evaluate model

The syntax conveniently follows that of sklearn.

In [25]:
from xgbsurv.datasets import load_metabric
from xgbsurv import XGBSurv
from sklearn.model_selection import train_test_split
import numpy as np
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [26]:
data, target = load_metabric(path="/Users/JUSC/Documents/xgbsurv/xgbsurv/datasets/data/", as_frame=False)
target_sign = np.sign(target)
X_train, X_test, y_train, y_test = train_test_split(data, target, stratify=target_sign)

## Load Model

In [27]:
model = XGBSurv(n_estimators=100, objective="ah_objective",
                                             eval_metric="ah_loss",
                                             learning_rate=0.3,
                                             random_state=7, 
                                             disable_default_eval_metric=True,
                                             base_score=0.0)

The options of loss and objective functions can be obtained like below:

In [28]:
print(model.get_loss_functions().keys())
print(model.get_objective_functions().keys())

dict_keys(['breslow_loss', 'efron_loss', 'cind_loss', 'deephit_loss', 'aft_loss', 'ah_loss'])
dict_keys(['breslow_objective', 'efron_objective', 'cind_objective', 'deephit_objective', 'aft_objective', 'ah_objective'])


## Fit Model

In [29]:
eval_set = [(X_train, y_train)]

In [30]:
model.fit(X_train, y_train, eval_set=eval_set)

[0]	validation_0-ah_likelihood:3.56551
[1]	validation_0-ah_likelihood:3.56516
[2]	validation_0-ah_likelihood:3.56481
[3]	validation_0-ah_likelihood:3.56447
[4]	validation_0-ah_likelihood:3.56412
[5]	validation_0-ah_likelihood:3.56377
[6]	validation_0-ah_likelihood:3.56342
[7]	validation_0-ah_likelihood:3.56307
[8]	validation_0-ah_likelihood:3.56272
[9]	validation_0-ah_likelihood:3.56238
[10]	validation_0-ah_likelihood:3.56203
[11]	validation_0-ah_likelihood:3.56168
[12]	validation_0-ah_likelihood:3.56133
[13]	validation_0-ah_likelihood:3.56099
[14]	validation_0-ah_likelihood:3.56064
[15]	validation_0-ah_likelihood:3.56029
[16]	validation_0-ah_likelihood:3.55994
[17]	validation_0-ah_likelihood:3.55960
[18]	validation_0-ah_likelihood:3.55925
[19]	validation_0-ah_likelihood:3.55890
[20]	validation_0-ah_likelihood:3.55856
[21]	validation_0-ah_likelihood:3.55821
[22]	validation_0-ah_likelihood:3.55786
[23]	validation_0-ah_likelihood:3.55751
[24]	validation_0-ah_likelihood:3.55717
[25]	valid

XGBSurv(base_score=0.0, booster=None, callbacks=None, colsample_bylevel=None,
        colsample_bynode=None, colsample_bytree=None,
        disable_default_eval_metric=True, early_stopping_rounds=None,
        enable_categorical=False,
        eval_metric=CPUDispatcher(<function ah_likelihood at 0x7fca085da9e0>),
        feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
        importance_type=None, interaction_constraints=None, learning_rate=0.3,
        max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
        max_delta_step=None, max_depth=None, max_leaves=None,
        min_child_weight=None, missing=nan, monotone_constraints=None,
        n_estimators=100, n_jobs=None, num_parallel_tree=None,
        objective=CPUDispatcher(<function ah_objective at 0x7fca085dac20>), ...)

The model can be saved like below. Note that objective and eval_metric are not saved.

## Predict

In [32]:
preds_train = model.predict(X_train, output_margin=True)
preds_test = model.predict(X_test, output_margin=True)

In [33]:
preds_test

array([-0.01782214, -0.02090764, -0.02114228, -0.02090764, -0.03134247,
       -0.03134247, -0.02800552, -0.03023037, -0.01791012, -0.02070156,
       -0.03134247, -0.02800552, -0.02269631, -0.03134247, -0.01407798,
       -0.03134247, -0.03023037, -0.01525407, -0.02090764, -0.01791012,
       -0.03134247, -0.01407798, -0.03134247, -0.03023037,  0.01735069,
       -0.01791012, -0.03134247, -0.01974595, -0.0220583 , -0.02090764,
       -0.02070156, -0.01791012, -0.0207073 , -0.02090764, -0.02800552,
       -0.03134247, -0.02090764, -0.02070156, -0.01782214, -0.02070156,
       -0.02090764, -0.02070156, -0.03134247, -0.022338  , -0.02040849,
       -0.01464348, -0.02057973, -0.02090764, -0.03134247, -0.03134247,
       -0.02800552, -0.02090764, -0.0207073 , -0.02336937, -0.02233403,
       -0.03134247, -0.02336937, -0.0220583 , -0.03134247, -0.02336937,
       -0.02070156, -0.00751907, -0.03134247, -0.03134247, -0.01464348,
       -0.03134247, -0.02233403, -0.02269631, -0.01782214, -0.03

## Evaluate

In [34]:
#from sksurv.metrics import concordance_index_censored
from xgbsurv.evaluation import cindex_censored, ibs

In [35]:
# train
cindex_censored(y_train, preds_train)

0.677029761702031

In [36]:
# test
cindex_censored(y_test, preds_test)

0.5699006436452799